- We want to plot arrow destinations between each timeframe depending on the start shift. 

In [1]:
%run ./Utility/akira.py

import pickle as pkl
import folium as f
import json
from pathlib import Path

PATH = './provided_files/Players/Markov'
X, Y = 87, 141

# to change later
INIT = "outputs_start8010_res30.pkl"

with open(f'{PATH}/{INIT}', 'rb') as fp:
    d = pkl.load(fp)
    
dest = d[0]
# del d
"""
The length of dest, avail and reward is 720, which corresponds to the maximum shift time possible (720 minutes / 12 hours).
-> These correspond to the time frame between each state at 1 minute intervals (columns of matrix).

Then, for each time frame there are 3508 possible cells to be in (rows of matrix)
"""

'\nThe length of dest, avail and reward is 720, which corresponds to the maximum shift time possible (720 minutes / 12 hours).\n-> These correspond to the time frame between each state at 1 minute intervals (columns of matrix).\n\nThen, for each time frame there are 3508 possible cells to be in (rows of matrix)\n'

In [2]:
board_df_file = Path("./data/shapefiles/game_board_2019.csv")

if not board_df_file.exists():
    # Create a csv for board
    if not board_df_file.parent.exists():
        board_df_file.parent.mkdir()
    with open(os.path.join(".", "provided_files", "shapefiles", "game_board_2019.geojson")) as fp:
        board = json.load(fp)

    board_df = pd.DataFrame(columns=("id", "coordinates", "neighbours"))
    for cell in board['features']:
        coordinates = np.array(cell['geometry']['coordinates'][0]).reshape(-1, 2).T
        cell_dict = dict(
            id = cell['properties']['id'],
            coordinates = coordinates,
            neighbours = cell['properties']['neighbours'],
            longitude_mean = np.mean(coordinates[0]),
            latitude_mean = np.mean(coordinates[1]),
        )
        board_df = board_df.append(cell_dict, ignore_index=True)
    board_df.set_index("id", inplace=True)
    board_df.to_csv(board_df_file)
else:
    board_df = pd.read_csv(board_df_file, index_col=0)

# clean up column names and generate dictionary to map idx to coordinates
board_df = board_df.reset_index()[['id','latitude_mean','longitude_mean']].rename({'latitude_mean': 'lat', 'longitude_mean': 'long'}, axis=1)
board_df['coord'] = list(zip(board_df['lat'],board_df['long']))
board_df.drop(['lat','long'], axis=1, inplace=True)

MAP = dict(zip(board_df['id'], board_df['coord']))

del board_df

In [3]:
def idx2int(t):
    return t[0] * 141 + t[1]

def int2idx(i):
    global Y
    return f"{i//Y}:{i%Y}"
    
def idx2coord(idx):
    global MAP
    return MAP[idx]

def convert(digit):
    global MAP
    return idx2coord(int2idx(digit))

def adjust(coord, gap=0.4):
    coord = np.array(coord)
    return tuple(coord[0]), tuple(coord[1] - gap * (coord[1] - coord[0]))

In [4]:
# (lat, long) for every destination if it is in a Manhattan cell
idx = {convert(i[0]): convert(i[1]) for i in dest[600].items()} # if i[0] in [idx2int(j) for j in M]}

def r(p1, p2):
    long_diff = np.radians(p2[1] - p1[1])
    
    lat1 = np.radians(p1[0])
    lat2 = np.radians(p2[0])
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    rotate = np.degrees(np.arctan2(x, y))
    if rotate < 0:
        return rotate + 360
    return rotate

m = f.Map(location=[40.783435, -73.96625], tiles="Stamen Terrain", zoom_start=11)

for coord in idx.items():
    if coord[0] == coord[1]:
        continue # no need to draw cell that corresponds to say
    coord = adjust(coord)
    f.PolyLine(locations=coord, weight=1, color = 'black').add_to(m)
    
    f.RegularPolygonMarker(location=coord[1], 
                          fill_color='black', number_of_sides=3, 
                          radius=3, rotation=r(coord[0], coord[1]) - 90, 
                           weight=0, fill=True, fill_opacity=1).add_to(m)

m.save("arrowmaps/sat600.html")

In [5]:
with open('provided_files/Players/Greedy/refinedManhattan.pickle', 'rb') as handle:
    M = pkl.load(handle)

with open('provided_files/Players/Greedy/refinedJFK.pickle', 'rb') as handle:
    J = pkl.load(handle)

with open('provided_files/Players/Greedy/refinedLaguardia.pickle', 'rb') as handle:
    L = pkl.load(handle)

In [6]:
with open(os.path.join(".", "provided_files", "shapefiles", "game_board_2019.geojson")) as fp:
    board = json.load(fp)
idToX = {}
idToY = {}
nbs = {}
for cell in board['features']:
    cellID = cell['properties']['id']
    cellTuple = tuple(map(int, cellID.split(':')))
    idToX[cellID], idToY[cellID] = cellTuple
    nbs[cellTuple] = [tuple(map(int, nb.split(':'))) for nb in cell['properties']['neighbours']]

In [7]:
cell = list(nbs.keys())[0]

In [8]:
greedy_dest = {}
for c in nbs:
    best = []
    for cell in nbs[c]:
        lowest_dist = (cell, min(M[cell], J[cell], L[cell]))
        best.append(lowest_dist)
    best = sorted(best, key=lambda x: x[1])[0][0]
    greedy_dest[f'{c[0]}:{c[1]}'] = f'{best[0]}:{best[1]}'

In [9]:
# (lat, long) for every destination if it is in a Manhattan cell
idx = {idx2coord(i[0]): idx2coord(i[1]) for i in greedy_dest.items()}

def r(p1, p2):
    long_diff = np.radians(p2[1] - p1[1])
    
    lat1 = np.radians(p1[0])
    lat2 = np.radians(p2[0])
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    rotate = np.degrees(np.arctan2(x, y))
    if rotate < 0:
        return rotate + 360
    return rotate

m = f.Map(location=[40.783435, -73.96625], tiles="Stamen Terrain", zoom_start=11)

for coord in idx.items():
    if coord[0] == coord[1]:
        continue # no need to draw cell that corresponds to say
    coord = adjust(coord)
    f.PolyLine(locations=coord, weight=1, color = 'black').add_to(m)
    
    f.RegularPolygonMarker(location=coord[1], 
                          fill_color='black', number_of_sides=3, 
                          radius=3, rotation=r(coord[0], coord[1]) - 90, 
                           weight=0, fill=True, fill_opacity=1).add_to(m)

m.save("arrowmaps/greedy.html")